This will handle the first step of the process - creating the code from the images.

In [11]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
from glob import glob
import private_secrets as secrets

In [12]:
# basic setup and auth
subscription_key = secrets.SUB_KEY
endpoint = "https://test-image-rec.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
GH_BASE = 'https://raw.githubusercontent.com/GitToby/college-code-reincarnation/master/meta_work/img/'

In [13]:
# fetch all images
images = glob("img/*")

# replace the folder to get just the image filenames
for i in range(len(images)):
    images[i] = images[i].replace("img/","")

image_urls = [GH_BASE + im for im in images]

print(len(images), "Images")

37 Images


In [14]:
def make_request(img_url: str, print_res: bool = False) -> str:
    # Make initial request
    recognize_handw_results = computervision_client.read(img_url, raw=True)

    # Parse response to get op ID 
    operation_location_remote = recognize_handw_results.headers["Operation-Location"]
    operation_id = operation_location_remote.split("/")[-1]
    
    # Await the response while remote does processing (nast impl but I dont care) 
    while True:
        get_handw_text_results = computervision_client.get_read_result(operation_id)
        if get_handw_text_results.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    # Print the detected text, line by line
    if get_handw_text_results.status == OperationStatusCodes.succeeded:
        for text_result in get_handw_text_results.analyze_result.read_results:
            if print_res:
                for line in text_result.lines:
                    print(line.text)
            return text_result.lines

In [15]:
def pipe_image_to_txt(image_file_path: str) -> (str,str):
    # form known url
    image_url = GH_BASE + image_file_path

    # Make request and get data
    res = make_request(image_url)

    # Join resulting data to a string
    out_text = "\n".join(t.text for t in res)
    
    # dump into an out file
    out_file_name = image_file_path.replace(".jpg",".txt")
    with open(f"out/{out_file_name}", "w") as f:
        f.write(out_text)

    # util result
    return image_file_path, out_file_name


In [41]:
for i, image_file_path in enumerate(sorted(images)):
    original, test_file = pipe_image_to_txt(image_file_path)
    print(i, "|", original, "->", test_file)

0 | addNewMember5.jpg -> addNewMember5.txt
1 | editMemberInfo3.jpg -> editMemberInfo3.txt
2 | sendEmail2.jpg -> sendEmail2.txt
3 | editMemberInfo1.jpg -> editMemberInfo1.txt
4 | addNewMember3.jpg -> addNewMember3.txt


ComputerVisionErrorException: Operation returned an invalid status code 'Bad Request'

In [40]:
files = glob("out/*")
f_map = dict()
for f in files:
    f_key = re.search("[a-zA-Z][0-9]",f).group()[:-1]
    if f_key in f_map.keys():
        f_map[f_key] += [f]
    else:
        f_map[f_key] = [f]

f_map

{'b': ['out/b1.txt', 'out/b2.txt'],
 'a': ['out/a1.txt', 'out/a3.txt', 'out/a2.txt']}

In [35]:
import re
re.search("[a-zA-Z][0-9]",files[0]).group()

'b1'